In [6]:
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def pickle_file(obj, path):
    file = open(path, 'wb')
    pickle.dump(obj, file)
    file.close()

def load_pickle(path):
    file = open(path, 'rb')
    obj = pickle.load(file)
    file.close()
    return obj

In [21]:
df = load_pickle('./data_merged/mta_broughs_cases')
df['WEEK_YEAR']= pd.to_datetime(df["DATE"].dt.strftime('%Y-%m'))
borough_dict = {'Bk': 'Brooklyn', 'Bx': 'Bronx', 'M': 'Manhattan', 'Q': 'Queens'}
df.head()

,DATE,DAILY_ENTRIES,DAILY_EXITS,LINE_DENSITY,Bk_DAILY_ENTRIES,Bx_DAILY_ENTRIES,M_DAILY_ENTRIES,Q_DAILY_ENTRIES,SI_DAILY_ENTRIES,Bk_DAILY_EXITS,...,probable_case_count,HOSPITALIZED_COUNT,DEATH_COUNT,DEATH_COUNT_PROBABLE,CASE_COUNT_7DAY_AVG,all_case_count_7day_avg,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,all_death_count_7day_avg,WEEK_YEAR
0,2020-02-29,1.098717e+07,1.014809e+07,6.651748e+06,1592354.0,472832.000000,8.103800e+06,8.138530e+05,4332.0,1300999.0,...,0,1,0,0,0,0,0,0,0,2020-02-01
1,2020-03-01,8.332771e+06,7.565224e+06,5.075832e+06,1202466.0,357483.000000,6.120865e+06,6.487580e+05,3199.0,1042235.0,...,0,1,0,0,0,0,0,0,0,2020-03-01
2,2020-03-02,2.107111e+07,1.813919e+07,1.258845e+07,2878680.0,877654.000000,1.571191e+07,1.575667e+06,27196.0,2171576.0,...,0,2,0,0,0,0,0,0,0,2020-03-01
3,2020-03-03,2.175668e+07,1.880443e+07,1.302943e+07,2940052.0,882679.553641,1.626314e+07,1.642746e+06,28062.0,2210979.0,...,0,7,0,0,0,0,0,0,0,2020-03-01
4,2020-03-04,2.183703e+07,1.891770e+07,1.302833e+07,2971407.0,892522.000000,1.629358e+07,1.652106e+06,27421.0,2264144.0,...,0,2,0,0,0,0,0,0,0,2020-03-01


In [23]:
def generate_figure(df, columns):
    # df = df[df.DATE.dt.weekday > 4]
    df = df.groupby('WEEK_YEAR').mean().reset_index()
    df['DATE'] = df.WEEK_YEAR
    # df = df[df.TIME_OF_WEEK == 'Weekend']
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'},
    #               yaxis_domain=[0, 0.94])

    fig.add_trace(go.Scatter(x=df.DATE, y=df.HOSP_COUNT_7DAY_AVG,
                        mode='lines',
                        name='Hospitalized',
                        opacity = .8))
    for col in columns:
        fig.add_trace(go.Scatter(x=df.DATE, y=df[col],
                            mode='lines',
                            name=borough_dict[col.split('_')[0]],
                            opacity = .35),
                            secondary_y = True)
    # fig.data[1].update(xaxis='x2')
    fig.update_yaxes(title_text="Hospitalized", secondary_y=False)
    fig.update_yaxes(title_text="Subway Traffic", secondary_y=True)
    fig.update_layout(title='Average Weekly Subway Use and Rates of Covid Hospitalization')
    
    return fig
fig = generate_figure(df, ['Bk_LINE_DENSITY', 'Bx_LINE_DENSITY', 'M_LINE_DENSITY','Q_LINE_DENSITY'])
fig.write_html("./images/Weekly_Subway_Use_By_Borough.html")
fig.show()
# generate_figure(df, ['Bk_LINE_DENSITY', 'Bx_LINE_DENSITY', 'Q_LINE_DENSITY']).show()

In [ ]:

# plt.figure(figsize=(4,12))

fig, ax = plt.subplots(figsize=(12,4))


# Twin the x-axis twice to make independent y-axes.
axes = [ax, ax.twinx(), ax.twinx()]

# Make some space on the right side for the extra y-axis.
fig.subplots_adjust(right=0.75)

# Move the last y-axis spine over to the right by 20% of the width of the axes
# axes[-1].spines['right'].set_position(('axes', 1.2))

# To make the border of the right-most axis visible, we need to turn the frame
# on. This hides the other plots, however, so we need to turn its fill off.
axes[-1].set_frame_on(True)
axes[-1].patch.set_visible(False)

# And finally we get to plot things...
colors = ('Red', 'Blue')
i=0
# df = df[df.DAILY_ENTRIES<1000000]
for ax, color in zip(axes, colors):
    if i ==0 :
        ax.plot(df.DATE,df.DAILY_EXITS, color=color)
        ax.set_ylabel('Daily Exits')
    else: 
        ax.plot(df.DATE,df.CASE_COUNT, color=color)
        ax.set_ylabel('Daily Cases')
    # ax.set_ylabel('%s Thing' % color, color=color)
    ax.tick_params(axis='y', colors=color)
    i+= 1
axes[0].set_xlabel('Date')


plt.show()

In [21]:
# df1 = df.melt(id_vars=['DATE']+list(df.keys()[5:]), var_name='Subway')
df['DAY_OF_WEEK'] = df.DATE.dt.day_name()
df['TIME_OF_WEEK'] = df.DATE.dt.weekday.replace([0,1,2,3,4],'Weekday').replace([5,6],'Weekend')
fig = px.line(df, x='DATE', y='DAILY_EXITS', color='TIME_OF_WEEK' )
# fig.add_trace()
fig.show()

In [119]:
# Create traces
# fig = go.Figure()
from plotly.subplots import make_subplots
def generate_figure(df):
    # df = df[df.TIME_OF_WEEK == 'Weekend']
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'},
    #               yaxis_domain=[0, 0.94])

    fig.add_trace(go.Scatter(x=df.DATE, y=df.HOSP_COUNT_7DAY_AVG,
                        mode='lines',
                        name='Hospitalized 7 Day Avg',
                        opacity = .8))
    fig.add_trace(go.Scatter(x=df.DATE[df.TIME_OF_WEEK == 'Weekend'], y=df.LINE_DENSITY[df.TIME_OF_WEEK == 'Weekend'],
                        mode='lines',
                        name='Weekday Subway Density (Sum)',
                        opacity = .5),
                        secondary_y = True)
    fig.add_trace(go.Scatter(x=df.DATE[df.TIME_OF_WEEK == 'Weekday'], y=df.LINE_DENSITY[df.TIME_OF_WEEK == 'Weekday'],
                        mode='lines',
                        name='Weekend Subway Density (Sum)',
                        opacity = .5),
                        secondary_y = True)
    # fig.data[1].update(xaxis='x2')
    fig.update_yaxes(title_text="Hospitalized", secondary_y=False)
    fig.update_yaxes(title_text="Subway Traffic", secondary_y=True)
    fig.update_layout(title='Subway Use and Rates of Covid Hospitalization')
    
    return fig
generate_figure(df).show()

In [8]:
df.columns

Index(['DATE', 'DAILY_ENTRIES', 'DAILY_EXITS', 'LINE_DENSITY',
       'Bk_DAILY_ENTRIES', 'Bx_DAILY_ENTRIES', 'M_DAILY_ENTRIES',
       'Q_DAILY_ENTRIES', 'SI_DAILY_ENTRIES', 'Bk_DAILY_EXITS',
       'Bx_DAILY_EXITS', 'M_DAILY_EXITS', 'Q_DAILY_EXITS', 'SI_DAILY_EXITS',
       'Bk_LINE_DENSITY', 'Bx_LINE_DENSITY', 'M_LINE_DENSITY',
       'Q_LINE_DENSITY', 'SI_LINE_DENSITY', 'DATE_OF_INTEREST', 'CASE_COUNT',
       'probable_case_count', 'HOSPITALIZED_COUNT', 'DEATH_COUNT',
       'DEATH_COUNT_PROBABLE', 'CASE_COUNT_7DAY_AVG',
       'all_case_count_7day_avg', 'HOSP_COUNT_7DAY_AVG',
       'DEATH_COUNT_7DAY_AVG', 'all_death_count_7day_avg'],
      dtype='object')

In [15]:

def generate_corr_table(full_df):
    corr_table = []
    rows = ['DAILY_ENTRIES', 'DAILY_EXITS', 'LINE_DENSITY']
    columns = ['CASE_COUNT', 'HOSPITALIZED_COUNT', 'DEATH_COUNT', 
        'CASE_COUNT_7DAY_AVG', 'all_case_count_7day_avg', 'HOSP_COUNT_7DAY_AVG',
        'DEATH_COUNT_7DAY_AVG', 'all_death_count_7day_avg']
    masks = ['Daily', 'Weekday', 'Weekend']

    full_df['TIME_OF_WEEK'] = full_df.DATE.dt.weekday.replace([0,1,2,3,4],'Weekday').replace([5,6],'Weekend')
    for mask in masks:
        if mask == 'Weekday':
            df = full_df[full_df.TIME_OF_WEEK == 'Weekday']
        elif mask == 'Weekend':
            df = full_df[full_df.TIME_OF_WEEK == 'Weekend']
        else:
            df = full_df
        for row in rows:
            dict_to_add = {'Mask':mask ,'Correlate':row}
            for column in columns:
                dict_to_add[column] = df[row].corr(df[column])
            corr_table.append(dict_to_add)
    return corr_table

def generate_corr_table_2(full_df):
    corr_table = []
    # rows = ['DAILY_ENTRIES', 'DAILY_EXITS', 'LINE_DENSITY']
    rows = ['Bk_LINE_DENSITY', 'Bx_LINE_DENSITY', 'M_LINE_DENSITY',
       'Q_LINE_DENSITY', 'SI_LINE_DENSITY']
    columns = ['CASE_COUNT', 'HOSPITALIZED_COUNT', 'DEATH_COUNT', 
        'CASE_COUNT_7DAY_AVG', 'all_case_count_7day_avg', 'HOSP_COUNT_7DAY_AVG',
        'DEATH_COUNT_7DAY_AVG', 'all_death_count_7day_avg']

    df = full_df
    for row in rows:
        dict_to_add = {'Correlate':row}
        for column in columns:
            dict_to_add[column] = df[row].corr(df[column])
        corr_table.append(dict_to_add)
    return corr_table
corr_table = pd.DataFrame(generate_corr_table_2(df))
# corr_table.to_csv('./tables/correlation_table_time_of_week.csv')
corr_table.head()


,Correlate,CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,CASE_COUNT_7DAY_AVG,all_case_count_7day_avg,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,all_death_count_7day_avg
0,Bk_LINE_DENSITY,0.029814,-0.301112,-0.412242,-0.043991,-0.036477,-0.343780,-0.418453,-0.409800
1,Bx_LINE_DENSITY,0.024843,-0.267438,-0.380951,-0.063265,-0.055679,-0.320002,-0.388803,-0.381172
2,M_LINE_DENSITY,0.041330,-0.289087,-0.388085,-0.022554,-0.016779,-0.326264,-0.392899,-0.381997
3,Q_LINE_DENSITY,0.048379,-0.309094,-0.438308,-0.028372,-0.019813,-0.355223,-0.445171,-0.435980
4,SI_LINE_DENSITY,0.047977,-0.187201,-0.277484,-0.033027,-0.029996,-0.233598,-0.284945,-0.276426


In [121]:
# corr_table.groupby(['Mask']).max()
corr_table = corr_table.round(3)
corr_table.set_index(['Correlate'], inplace=True)

# corr_table.to_markdown('./tables/correlation_table_time_of_week.md')
corr_table.head()

,CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,CASE_COUNT_7DAY_AVG,all_case_count_7day_avg,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,all_death_count_7day_avg
Correlate,,,,,,,,
DAILY_ENTRIES,0.019,-0.287,-0.392,-0.055,-0.049,-0.332,-0.399,-0.388
DAILY_EXITS,0.052,-0.305,-0.414,-0.013,-0.006,-0.342,-0.418,-0.409
LINE_DENSITY,0.035,-0.299,-0.408,-0.037,-0.030,-0.341,-0.414,-0.404


In [74]:
corr_table.abs().sort_values(('Daily', 'LINE_DENSITY'),axis=1, ascending=False)

DEATH_COUNT_7DAY_AVG  DEATH_COUNT  \
Mask    Correlate                                          
Daily   DAILY_ENTRIES                 0.399        0.392   
        DAILY_EXITS                   0.418        0.414   
        LINE_DENSITY                  0.414        0.408   
Weekday DAILY_ENTRIES                 0.463        0.458   
        DAILY_EXITS                   0.466        0.463   
        LINE_DENSITY                  0.478        0.474   
Weekend DAILY_ENTRIES                 0.488        0.481   
        DAILY_EXITS                   0.472        0.468   
        LINE_DENSITY                  0.497        0.492   

                       all_death_count_7day_avg  HOSP_COUNT_7DAY_AVG  \
Mask    Correlate                                                      
Daily   DAILY_ENTRIES                     0.388                0.332   
        DAILY_EXITS                       0.409                0.342   
        LINE_DENSITY                      0.404                0.341   
Weekday DAILY_ENTRIES                     0.451                0.382   
        DAILY_EXITS                       0.456                0.378   
        LINE_DENSITY                      0.467                0.391   
Weekend DAILY_ENTRIES                     0.473                0.421   
        DAILY_EXITS                       0.459                0.397   
        LINE_DENSITY                      0.483                0.424   

                       HOSPITALIZED_COUNT  CASE_COUNT_7DAY_AVG  CASE_COUNT  \
Mask    Correlate                                                            
Daily   DAILY_ENTRIES               0.287                0.055       0.019   
        DAILY_EXITS                 0.305                0.013       0.052   
        LINE_DENSITY                0.299                0.037       0.035   
Weekday DAILY_ENTRIES               0.357                0.056       0.028   
        DAILY_EXITS                 0.361                0.007       0.017   
        LINE_DENSITY                0.370                0.034       0.009   
Weekend DAILY_ENTRIES               0.382                0.110       0.088   
        DAILY_EXITS                 0.368                0.046       0.036   
        LINE_DENSITY                0.390                0.082       0.067   

                       all_case_count_7day_avg  
Mask    Correlate                               
Daily   DAILY_ENTRIES                    0.049  
        DAILY_EXITS                      0.006  
        LINE_DENSITY                     0.030  
Weekday DAILY_ENTRIES                    0.048  
        DAILY_EXITS                      0.001  
        LINE_DENSITY                     0.027  
Weekend DAILY_ENTRIES                    0.101  
        DAILY_EXITS                      0.037  
        LINE_DENSITY                     0.073

In [29]:
print(f'Correlation between Subway Entries and Covid Cases:\t{df.DAILY_ENTRIES.corr(df.HOSP_COUNT_7DAY_AVG)}')
print(f'Correlation between Subway Exits and Covid Cases:\t{df.DAILY_EXITS.corr(df.HOSP_COUNT_7DAY_AVG)}')
print(f'Correlation between Line Density and Covid Cases:\t{df.LINE_DENSITY.corr(df.HOSP_COUNT_7DAY_AVG)}')

Correlation between Subway Entries and Covid Cases:	-0.4210110108217203
Correlation between Subway Exits and Covid Cases:	-0.3974747024305076
Correlation between Line Density and Covid Cases:	-0.42385720942325505


In [22]:
df = df[df.TIME_OF_WEEK == 'Weekend']

In [27]:
for x in range(-14,0):
    print(f'Correlation between Line Density and Covid Cases {-x} days later:\t{df.DAILY_EXITS.shift(x).corr(df.CASE_COUNT.iloc[:x])}')

Correlation between Line Density and Covid Cases 14 days later:	0.0770438946965025
Correlation between Line Density and Covid Cases 13 days later:	0.11008927080244933
Correlation between Line Density and Covid Cases 12 days later:	0.04216675584115764
Correlation between Line Density and Covid Cases 11 days later:	0.08459921553077798
Correlation between Line Density and Covid Cases 10 days later:	0.0277357720267018
Correlation between Line Density and Covid Cases 9 days later:	0.030059785495632107
Correlation between Line Density and Covid Cases 8 days later:	-0.008630599965366697
Correlation between Line Density and Covid Cases 7 days later:	-0.020227781505678662
Correlation between Line Density and Covid Cases 6 days later:	-0.03774931143740905
Correlation between Line Density and Covid Cases 5 days later:	-0.013752864111256443
Correlation between Line Density and Covid Cases 4 days later:	-0.05333670330824325
Correlation between Line Density and Covid Cases 3 days later:	-0.032622742

In [28]:
for x in range(1,14):
    print(f'Correlation between Line Density and Covid Cases {x} days before:\t{df.DAILY_EXITS.shift(x).corr(df.CASE_COUNT.iloc[x:])}')

Correlation between Line Density and Covid Cases 1 days before:	-0.0480335391831696
Correlation between Line Density and Covid Cases 2 days before:	0.015330823119700736
Correlation between Line Density and Covid Cases 3 days before:	0.029487996940136263
Correlation between Line Density and Covid Cases 4 days before:	0.10747593897455375
Correlation between Line Density and Covid Cases 5 days before:	0.16931407953418942
Correlation between Line Density and Covid Cases 6 days before:	0.20012947705495013
Correlation between Line Density and Covid Cases 7 days before:	0.266882193491665
Correlation between Line Density and Covid Cases 8 days before:	0.25096780051565964
Correlation between Line Density and Covid Cases 9 days before:	0.3011945318840952
Correlation between Line Density and Covid Cases 10 days before:	0.26757720711799177
Correlation between Line Density and Covid Cases 11 days before:	0.30726681310921167
Correlation between Line Density and Covid Cases 12 days before:	0.27786402